In [1]:
import sys
sys.path.append('../src')

import numpy as np
import joblib
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import scipy.optimize

import simulation

In [2]:
cerro_toco_atm_profile = np.loadtxt('../input-data/cerro_toco_atm_profile.txt')

In [3]:
mag_vec = SkyCoord(-5.9, 21.2, frame='altaz', unit='deg')
bc = 38.0
bw = 5.7

In [4]:
def run_calc(mag_diff, zenith):
    simulator = simulation.AtmSim(22738, cerro_toco_atm_profile, 5200)
    return simulator.calc_propagation_matrix_avg_band(bc - bw, bc + bw, 25, mag_diff, zenith)[3]

In [5]:
az = np.arange(0, 360, 10)
alt = np.linspace(30, 60, 13)
alt_az = np.array(np.meshgrid(alt, az)).T
alt_az = alt_az.reshape((alt_az.shape[0] * alt_az.shape[1], 2))

obs_directions = SkyCoord(az=alt_az[:, 1], alt=alt_az[:, 0], frame="altaz", unit="deg")
mag_diffs = mag_vec.separation(obs_directions).rad
zeniths = np.deg2rad(90 - alt_az[:, 0])

grid_scan = np.array(joblib.Parallel(n_jobs=10)(joblib.delayed(run_calc)(mag_diffs[i], zeniths[i]) for i in range(mag_diffs.size)))

In [6]:
def fit(x, a, b, c, d):
    zenith = np.pi / 2 - x[0]
    az = x[1]
    return a * np.tan(b * zenith) * np.cos(az - c) + d
fit_results = scipy.optimize.curve_fit(fit, np.deg2rad(alt_az.T), grid_scan)

In [7]:
fit_results

(array([ 1.10639594e-04,  9.84847471e-01, -1.02974426e-01,  4.18500536e-05]),
 array([[ 5.58519928e-15, -2.58829172e-11,  1.01329809e-19,
         -1.02951228e-26],
        [-2.58829172e-11,  1.24284560e-07, -5.11629354e-17,
          1.38078722e-22],
        [ 1.01329809e-19, -5.11629354e-17,  1.59254004e-08,
          2.20757169e-19],
        [-1.02951228e-26,  1.38078722e-22,  2.20757169e-19,
          1.14521333e-16]]))

In [8]:
np.save('../result-data/fit-results.npy', fit_results[0])

In [9]:
diffs = fit(np.deg2rad(alt_az.T), *fit_results[0]) - grid_scan
print(f'maximum residual: {np.max(np.abs(diffs)) * 1e9:.0f} nK')
print(f'average residual: {np.mean(np.abs(diffs)) * 1e9:.0f} nK')

maximum residual: 472 nK
average residual: 196 nK
